In [2]:
!pip install --ignore-installed pycaret[full]
!pip install python-dotenv

  Obtaining dependency information for pycaret[full] from https://files.pythonhosted.org/packages/d5/54/d575af389203fc27d6c6cf7d60c4e67fcabfda4bc8e84271c8a396bd4a03/pycaret-3.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for ipython>=5.5.0 from https://files.pythonhosted.org/packages/ef/02/fc039fca3ec40a00f962eb6e9da45c507334b0650a3cb9facd38d234fb7a/ipython-8.16.1-py3-none-any.whl.metadata
  Obtaining dependency information for ipywidgets>=7.6.5 from https://files.pythonhosted.org/packages/4a/0e/57ed498fafbc60419a9332d872e929879ceba2d73cb11d284d7112472b3e/ipywidgets-8.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for tqdm>=4.62.0 from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 71.1 MB/s eta 0:00:0000:0100:01
   

In [6]:
pip install s3fs

Note: you may need to restart the kernel to use updated packages.


In [1]:
# import libraries
import boto3, os
from sagemaker import get_execution_role
import pandas as pd
from dotenv import load_dotenv
from load_data import load_data



my_region = boto3.session.Session().region_name

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/lanre.bakare/Library/Application Support/sagemaker/config.yaml


In [2]:
load_dotenv(".env")
role = get_execution_role()

data_location_s3 = os.getenv("data_location_s3")
target = os.getenv("target")
data_location = 's3://{}'.format(data_location_s3)
print(data_location)
print(target)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/lanre.bakare/Library/Application Support/sagemaker/config.yaml
s3://streaming-data-platform-ml-data/bakerloo.csv
Bakerloo10


In [3]:
data = load_data(data_location)
df = data.copy()
df.shape

(7127, 5)

In [4]:
print(df.shape)
# Randomly shuffle the DataFrame
df_shuffled = df.sample(frac=1).reset_index(drop=True)
# Sort by day and then pick the first 80% as your test data. 
train_size = int(0.8 * len(df))
train_data = df_shuffled[:train_size]
test_data = df_shuffled[train_size:]
print(train_data.shape, test_data.shape)

(7127, 5)
(5701, 5) (1426, 5)


In [5]:
from pycaret.regression import *
s = setup(data = train_data, target = target, session_id=123)

,Description,Value
0,Session id,123
1,Target,Bakerloo10
2,Target type,Regression
3,Original data shape,"(5701, 5)"
4,Transformed data shape,"(5701, 5)"
5,Transformed train set shape,"(3990, 5)"
6,Transformed test set shape,"(1711, 5)"
7,Numeric features,4
8,Preprocess,True
9,Imputation type,simple


In [6]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.7246,1.2800,1.1156,0.9769,0.0728,0.0522,0.0840
lightgbm,Light Gradient Boosting Machine,0.7430,1.3573,1.1493,0.9755,0.0773,0.0537,0.3840
xgboost,Extreme Gradient Boosting,0.7213,1.3654,1.1560,0.9754,0.0747,0.0498,0.0120
et,Extra Trees Regressor,0.6247,1.3690,1.1542,0.9753,0.0686,0.0379,0.0390
rf,Random Forest Regressor,0.6719,1.3808,1.1630,0.9751,0.0686,0.0408,0.0510
gbr,Gradient Boosting Regressor,0.8856,1.7308,1.3073,0.9689,0.0942,0.0696,0.0240
knn,K Neighbors Regressor,0.9112,1.8706,1.3553,0.9663,0.0856,0.0599,0.0060
dt,Decision Tree Regressor,0.7450,2.3048,1.5066,0.9583,0.0866,0.0430,0.0060
lar,Least Angle Regression,1.1447,2.6428,1.6204,0.9527,0.1387,0.0995,0.0040
lr,Linear Regression,1.1447,2.6428,1.6204,0.9527,0.1387,0.0995,0.2760


In [7]:
# Users are allowed to create their own model if they want to.
# This uses the default hyperparameters to train the model which you can then choose to tune as you want.
# In order to tune hyperparameters, the tune_model function is used
xgboost = create_model('xgboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.7305,1.2980,1.1393,0.9732,0.0640,0.0440
1,0.7220,1.4711,1.2129,0.9752,0.0781,0.0473
2,0.7989,1.9939,1.4120,0.9646,0.0786,0.0506
3,0.7337,1.3337,1.1548,0.9759,0.0732,0.0484
4,0.7200,1.2655,1.1250,0.9764,0.0635,0.0468
5,0.6876,0.9297,0.9642,0.9857,0.0716,0.0553
6,0.7160,1.2681,1.1261,0.9790,0.0756,0.0527
7,0.6585,0.9319,0.9654,0.9835,0.0860,0.0520
8,0.6886,0.9434,0.9713,0.9835,0.0670,0.0476


In [8]:
tuned_model = tune_model(xgboost)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.7882,1.4068,1.1861,0.9710,0.0698,0.0468
1,0.7249,1.2204,1.1047,0.9795,0.0790,0.0476
2,0.8239,2.0706,1.4389,0.9632,0.0794,0.0531
3,0.7300,1.2804,1.1316,0.9768,0.0716,0.0463
4,0.7069,1.1199,1.0583,0.9791,0.0606,0.0445
5,0.6786,0.9620,0.9808,0.9852,0.0627,0.0486
6,0.7037,1.0165,1.0082,0.9832,0.0734,0.0504
7,0.6665,0.7817,0.8842,0.9862,0.0675,0.0475
8,0.6989,0.8839,0.9402,0.9845,0.0638,0.0467


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [9]:
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [10]:
predict_model(best)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,0.7223,1.2046,1.0975,0.9781,0.0668,0.0477


,Bakerloo,dayOfWeek,hour,minute,Bakerloo10,prediction_label
2676,24,1,19,53,23.0,23.593157
724,21,3,21,53,22.0,21.209566
4595,23,4,6,55,23.0,23.379546
1112,24,1,10,54,25.0,23.443638
4064,17,1,5,26,14.0,14.129906
...,...,...,...,...,...,...
3854,23,0,13,21,19.0,22.516597
4481,15,0,23,4,18.0,17.425135
467,2,6,3,24,2.0,1.831999
3551,22,4,10,43,21.0,21.709514


In [11]:
final_best_model = finalize_model(best)
print(final_best_model)

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Bakerloo', 'dayOfWeek', 'hour',
                                             'minute'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('actual_estimator',
                 <catboost.core.CatBoostRegressor object at 0x2adb69f60>)])


In [12]:
final_prediction = predict_model(final_best_model, data=test_data)
final_prediction.head()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,0.7139,1.3315,1.1539,0.9781,0.0705,0.0517


,Bakerloo,dayOfWeek,hour,minute,Bakerloo10,prediction_label
5701,27,1,7,50,27.0,25.697925
5702,21,4,13,5,21.0,21.030478
5703,21,3,21,25,21.0,20.924487
5704,25,2,7,55,26.0,25.706559
5705,23,0,9,58,21.0,22.928584


In [13]:
from pycaret.utils.generic import check_metric
check_metric(final_prediction.Bakerloo10, final_prediction.prediction_label, 'RMSE')

1.1539